# TopBrass Profit Optimization with GAMSpy

This notebook solves a simple linear programming problem using `GAMSpy`. The goal is to find the optimal amount of football soccer trophies to produce to maximize profits.

## Problem
* **Football Trophies** (var `f`) are larger in size so they require 4 sq. feet of wood on their base and yield a profit of $12
* **Soccer Trophies** (var `s`) are smaller so they need 2 sq. feet of wood on the base and yield a smaller profit of $9.

## Constraints
1. There are only 1000 brass footballs
2. There are only 1500 soccer balls
3. There are 1750 total plaques available
4. There's only 4800 sq. feet of wood available

**Objective:** Find the production of football trophies and soccer trophies that will yield the most profit for TopBrass

In [2]:
import gamspy as gp

## Step 1. Set up the Model

In [9]:
m = gp.Container()

# Variables are positive because you can't produce a negative amount of trophies
f = gp.Variable(m, 'f', 'positive')
s = gp.Variable(m, 's', 'positive')
# adding .up to replace the upper limit equations for the variables
f.up = 1000
s.up = 1500

# profit will be p
p = gp.Variable(m, 'p', 'free')

### Defining the Equations (constraints and objective)

Now it's time to translate the constraints and profit equations. There are a total of 4 constraints and 1 objective for this problem. 

In [10]:
# Constraint 1: Only have 1750 plaques for all of the trophies
c1 = gp.Equation(m, 'constraint3')
c1[:] = f + s <= 1750

# Constraint 4: Only have 4800 sq. feet of wood to use
c2 = gp.Equation(m, 'constraint4')
c2[:] = 4*f + 2*s <= 4800

# Objective: This is what defines the total profits
obj = gp.Equation(m, 'defobj')
obj[:] = p == 12*f + 9*s

## Step. 2: Solving the Model

In [11]:
topBrass = gp.Model(
    m,
    name= 'topBrass',
    equations= [c1, c2, obj],
    objective= p,
    sense = 'max',
    problem = 'lp',)

topBrass.solve()

,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,OptimalGlobal,17700.0,3,3,LP,CPLEX,0.0


### IMPORTANT

Now that I'm using .up, the upper value is no longer inf. Also notice that the variables are presented like databases.

In [12]:
display(f.records, s.records, p.records)

,level,marginal,lower,upper,scale
0,650.0,0.0,0.0,1000.0,1.0


,level,marginal,lower,upper,scale
0,1100.0,0.0,0.0,1500.0,1.0


,level,marginal,lower,upper,scale
0,17700.0,0.0,-inf,inf,1.0


In [ ]:
# another way to check the the values in the var
p.toValue()

np.float64(17700.0)